In [29]:
# import necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels import regression

D:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
D:\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
# to remind us the 11 sectors of S&P500
symbols_index = {"XLB":"Materials Select Sector SPDR Fund",
                "XLE":"Energy Select Sector SPDR Fund",
                "XLF":"Financial Select Sector SPDR Fund",
                "XLI":"Industrial Select Sector SPDR Fund",
                "XLK":"Technology Select Sector SPDR Fund",
                "XLP":"Consumer Staples Select Sector SPDR Fund",
                "XLRE":"Real Estate Select Sector SPDR Fund",
                "XLU":"Utilities Select Sector SPDR Fund",
                "XLV":"Health Care Select Sector SPDR Fund",
                "XLY":"Consumer Discretionary Select Sector SPDR Fund",
                "XTL":"Telecommunication Select Sector SPDR Fund",
                "SPY":"SPDR S&P 500 ETF Trust",
                "^GSPC":"S&P 500 Index"}

In [11]:
# read 11 sectors' data from csv. It is too much data to request if we use api. 
# And it will reach the limit of requests per day of api. So we decide to use csv instead in this notebook.
sector_df_origin = dict()
sector_df = dict()
for key in symbols_index:
    df_temp = pd.read_csv(key+".csv", parse_dates = True, index_col = "Date")
    sector_df_origin[key] = df_temp
    df_temp_copy = df_temp.copy()
    df_temp_copy["return"] = df_temp_copy["Adj Close"].pct_change()
    sector_df[key] = df_temp_copy.iloc[1:]

In [12]:
# check the dataframe
sector_df_origin["XLB"].head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2015-06-19,50.470001,50.639999,50.209999,50.209999,45.504456,3697500
2015-06-22,50.480000,50.480000,50.250000,50.400002,45.676640,1981300
2015-06-23,50.419998,50.480000,50.160000,50.259998,45.549763,2251600
2015-06-24,50.150002,50.180000,49.610001,49.619999,44.969738,3141300
2015-06-25,49.639999,49.820000,49.430000,49.580002,44.933491,2448100


In [13]:
sector_df["^GSPC"].head()

,Open,High,Low,Close,Adj Close,Volume,return
Date,,,,,,,
2015-06-22,2112.500000,2129.870117,2112.500000,2122.850098,2122.850098,3030020000,0.006095
2015-06-23,2123.159912,2128.030029,2119.889893,2124.199951,2124.199951,3091190000,0.000636
2015-06-24,2123.649902,2125.100098,2108.580078,2108.580078,2108.580078,3102480000,-0.007353
2015-06-25,2109.959961,2116.040039,2101.780029,2102.310059,2102.310059,3214610000,-0.002974
2015-06-26,2102.620117,2108.919922,2095.379883,2101.489990,2101.489990,5025470000,-0.000390


In [15]:
sector_df["^GSPC"].index[1]

Timestamp('2015-06-23 00:00:00')

In [17]:
# define a function to find the trading dates that close to the date we decided
def find_tading_date(DataFrame, start_date, end_date):
    start_date = pd.Timestamp(start_date)
    end_date = pd.Timestamp(end_date)
    while start_date not in DataFrame.index:
        start_date = start_date - pd.Timedelta(1,"D")
    while end_date not in DataFrame.index:
        end_date = end_date - pd.Timedelta(1,"D")
    print(start_date, end_date)
    return start_date,end_date

In [19]:
# apply the function to find trading date
start_date = "2020-02-18"
end_date = "2020-06-18"
start_date, end_date = find_tading_date(sector_df["^GSPC"],start_date = start_date, end_date = end_date)

2020-02-18 00:00:00 2020-06-18 00:00:00


In [20]:
end_date - start_date

Timedelta('121 days 00:00:00')

In [30]:
# define a function to calculate the beta value
def linreg(x,y):
    x = sm.add_constant(x)
    model = sm.OLS(y,x).fit()
    return model.params[1]

In [50]:
sector_df.keys()

dict_keys(['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY', 'XTL', 'SPY', '^GSPC'])

In [59]:
# apply linear regresssion with specific x and y to calculate the beta value of each sector
y_list = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLRE', 'XLU', 'XLV', 'XLY', 'XTL']
x_list = ['SPY', '^GSPC']
beta_dict = dict()
for y in y_list:
    for x in x_list:
        beta = linreg(sector_df[x]["return"].loc[start_date:end_date].values,
                      sector_df[y]["return"].loc[start_date:end_date].values)
        if y not in beta_dict.keys():
            beta_dict[y] = {x:beta}
        else:
            beta_dict[y][x] = beta

In [66]:
# define a function to apply linear regression for later use
def multiple_beta_builder(x_list, y_list, start_date, end_date):
    beta_dict = dict()
    for x in x_list:
        for y in y_list:
            beta = linreg(sector_df[x]["return"].loc[start_date:end_date].values,
                          sector_df[y]["return"].loc[start_date:end_date].values)
            if x not in beta_dict.keys():
                beta_dict[x] = {y:beta}
            else:
                beta_dict[x][y] = beta
    beta_df = pd.DataFrame(beta_dict)
    return beta_df

In [67]:
# beta values of each sector after COVID-19 took place
COVID_beta_df = multiple_beta_builder(x_list, y_list, start_date = start_date, end_date = end_date)
COVID_beta_df

,SPY,^GSPC
XLB,1.071260,1.033130
XLE,1.411604,1.354302
XLF,1.271558,1.228168
XLI,1.117468,1.076040
XLK,1.119791,1.085400
XLP,0.761840,0.738244
XLRE,1.116072,1.081173
XLU,1.027350,0.996603
XLV,0.851600,0.825683
XLY,0.971094,0.937087


In [72]:
print("Largest beta during COVID-19: XLE," , symbols_index["XLE"] , COVID_beta_df["^GSPC"].loc["XLE"])
print("Smallest beta during COVID-19: XLP," , symbols_index["XLP"], COVID_beta_df["^GSPC"].loc["XLP"])

Largest beta during COVID-19: XLE, Energy Select Sector SPDR Fund 1.3543016374507317
Smallest beta during COVID-19: XLP, Consumer Staples Select Sector SPDR Fund 0.7382437522665167


In [68]:
# retrodict a date as the start date of the time period before COVID-19 according to the days  had passed since COVID-19 started.
preCOVID_start = "2019-10-18"
preCOVID_end = start_date
preCOVID_start, preCOVID_end = find_tading_date(sector_df["^GSPC"], preCOVID_start, preCOVID_end)

2019-10-18 00:00:00 2020-02-18 00:00:00


In [69]:
# beta values of each sectors before COVID-19
preCOVID_beta_df = multiple_beta_builder(x_list, y_list, start_date = preCOVID_start, end_date = preCOVID_end)
preCOVID_beta_df

,SPY,^GSPC
XLB,1.130307,1.139587
XLE,1.186348,1.188401
XLF,1.094992,1.100635
XLI,1.154670,1.159320
XLK,1.469072,1.475339
XLP,0.311518,0.321827
XLRE,0.329272,0.344970
XLU,-0.012803,0.001740
XLV,0.883104,0.886495
XLY,0.816509,0.826018


In [74]:
print("Largest beta before COVID-19: XLK," , symbols_index["XLK"] , preCOVID_beta_df["^GSPC"].loc["XLK"])
print("Smallest beta before COVID-19: XLU," , symbols_index["XLU"], preCOVID_beta_df["^GSPC"].loc["XLU"])

Largest beta before COVID-19: XLK, Technology Select Sector SPDR Fund 1.4753390191319242
Smallest beta before COVID-19: XLU, Utilities Select Sector SPDR Fund 0.0017395259672557117


In [77]:
# the differences between beta values before and during COVID-19
beta_difference_df = COVID_beta_df - preCOVID_beta_df
beta_difference_df

,SPY,^GSPC
XLB,-0.059047,-0.106456
XLE,0.225257,0.165900
XLF,0.176566,0.127533
XLI,-0.037202,-0.083280
XLK,-0.349282,-0.389939
XLP,0.450323,0.416417
XLRE,0.786801,0.736203
XLU,1.040152,0.994864
XLV,-0.031504,-0.060811
XLY,0.154585,0.111069


In [78]:
print("Largest positive beta difference: XLU," , symbols_index["XLU"] , beta_difference_df["^GSPC"].loc["XLU"])
print("Largest negative beta difference: XLK," , symbols_index["XLK"], beta_difference_df["^GSPC"].loc["XLK"])
print("Smallest beta difference: XTL," , symbols_index["XTL"], beta_difference_df["^GSPC"].loc["XTL"])

Largest positive beta difference: XLU, Utilities Select Sector SPDR Fund 0.9948635707689599
Largest negative beta difference: XLK, Technology Select Sector SPDR Fund -0.389939485460695
Smallest beta difference: XTL, Telecommunication Select Sector SPDR Fund -0.05517186756034398
